### Redes de classificação
Selecione um dataset e crie uma rede de classificação. Avalie o resultado com duas configurações diferentes. As configurações devem mudar no mínimo 3 hiperparâmetros (por exemplo, parâmetros do optimizer,  loss, número de camadas, função de ativação, normalização).

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torchvision.models import MobileNet_V3_Small_Weights
from torch.utils.data import DataLoader

In [5]:
weights = MobileNet_V3_Small_Weights.DEFAULT

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=weights.transforms())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=weights.transforms())

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:14<00:00, 11.8MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)

c:\Users\dvccs\AppData\Local\pypoetry\Cache\virtualenvs\vcomp-TtMCa82b-py3.12\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\dvccs\AppData\Local\pypoetry\Cache\virtualenvs\vcomp-TtMCa82b-py3.12\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to C:\Users\dvccs/.cache\torch\hub\checkpoints\mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 38.4MB/s]


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Device: {device}")

Device: cuda


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0)